#Install

In [ ]:
!pip install python-pptx

from google.colab import drive
import os
import pptx
from pptx.enum.shapes import MSO_SHAPE_TYPE

!pip install openai
import openai
import os
import time
import pandas as pd
!pip install --upgrade openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 38.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.7/152.7 kB 7.9 MB/s eta 0:00:00
  Created wheel for python-pptx: filename=python_pptx-0.6.21-py3-none-any.whl size=470949 sha256=27ea170f3ba7f2f5b4f590b56d2f5839e1d94150f6b854082168d4f56b2251aa
  Stored in directory: /root/.cache/pip/wheels/0e/4a/ed/9653bc799915f52dce3f04d14946fbd85cce9c3cdedc9cfa71
Successfully built python-pptx
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB

In [ ]:
#your API key
os.environ["OPENAI_API_KEY"] = '[YOUR_API_KEY]'

#load ppt from the google drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/analyzer/input_ppt")

Mounted at /content/gdrive


# Functions

In [26]:
class ChatApp: #no demo, only template
    def __init__(self, input):
        # Setting the API key to use the OpenAI API
        openai.api_key = os.getenv("OPENAI_API_KEY")
        self.messages = input

    def chat(self, message):
        self.messages.append({"role": "user", "content": message})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=self.messages,
            temperature=0.7,
            top_p = 1,
        )
        self.messages.append({"role": "assistant", "content": response["choices"][0]["message"].content})

        for i in range(2):
          self.messages.pop()

        return response["choices"][0]["message"]["content"]

In [27]:
def make_prompt_format(system, user, assistant): #prompt demonstration
  #system
  input = [{"role": "system", "content": system}]
  if user != "":
    #user
    demo_user = []
    demo_user.append(user)
    #assistant
    demo_assistant = []
    demo_assistant.append(assistant)

    #make format for API
    demo = pd.DataFrame({'user': demo_user, 'assistant': demo_assistant})
    for prompt, completion in zip(demo['user'], demo['assistant']):
      message_dict = {"role": "user", "content": prompt}
      input.append(message_dict)
      message_dict = {"role": "assistant", "content": completion}
      input.append(message_dict)
    
  return input

#Main

##Load Data

In [28]:
#load ppt file
ppt = pptx.Presentation('Architecture_Design.pptx')

#full slide
full_slide = []
content_full = "Entire contents: "
for i, slide in enumerate(ppt.slides):
  content_full = content_full + f"  *slide {i}: "
  for shape in slide.shapes:
    if hasattr(shape, "text"):
      content_full = content_full + shape.text
full_slide = content_full

# print("*"*10)
# print(full_slide)

#each slide, keywords
each_slide = {} #dictionary form
for i, slide in enumerate(ppt.slides):
  content_each = ""
  for shape in slide.shapes:
    if hasattr(shape, "text"):
      content_each = content_each + shape.text
  each_slide[f'{i}'] = content_each 

# print("*"*10)
# print(each_slide)

**********
Entire contents:   *slide 0: Architectural Design  *slide 1: Architectural Design …Architectural design identifies the main structural components in a system and the relationships between them. 
The output of this phase is an architectural model that describes how the system is organized as a set of communicating components. 

The architecture of a software system has significant impact on system performance, efficiency, security, and maintainability.
Architectural design flaws may result in project failure! 
  *slide 2: Design flaws – a few examplesWhat if the system does not handle multiple customers purchasing items simultaneously ?

What if web server does too much processing for each transaction? 

What if the security requirements are ignored ? 

What if the team misses to identify several generic components for which COTS are available ?

  *slide 3: Design flaws – a few examplesWhat if the system does not handle multiple customers purchasing items simultaneously ?


##Full Slide

In [29]:
#prompt demonstration
full_system = "I want you to summarize the given content. I will provide you with the entire contents of a PowerPoint presentation, and you will provide me with a summary of the entire slides."
full_user = ""
full_assistant = ""
prompt_full = make_prompt_format(full_system, full_user, full_assistant)

app_full = ChatApp(prompt_full)
message = full_slide
result_full = app_full.chat(message)

result_full

'The PowerPoint presentation covers the topic of Architectural Design and its importance in software development. It begins by defining Architectural Design and highlighting its significance in terms of system performance, efficiency, security, and maintainability. The presentation then proceeds to discuss the consequences of architectural design flaws, providing examples of such flaws. It talks about the architectural design process, objectives, and decisions, focusing on maintainability, use of commercial off-the-shelf (COTS) parts, system performance, security, safety, availability, reliability, fault tolerance, and recovery. The presentation explains four types of systems/subsystems, namely interactive systems, event-driven systems, transformational systems, and database systems, and provides examples for each. It also discusses different software architecture styles, such as client-server architecture, N-tier or multi-tier architecture, main program and subroutines architecture, e

##Each Slide

In [33]:
#prompt demonstration
each_system = "I want you to summarize the given content. I will provide you with the contents of a PowerPoint presentation, and you will provide me with a summary of the slide."
each_user = "Design flaws – a few examples What if the system does not handle multiple customers purchasing items simultaneously ?\n\t\uf0e0the shopping cart becomes common for both customers! \uf04c\n\nWhat if web server does too much processing for each transaction? \n\t\uf0e0capacity of the web server comes down drastically & the web server crashes when too many requests come within a small-time window!\n\nWhat if the security requirements are ignored ? \n\t\uf0e0hacker can manipulate the prices of items in the shopping cart \n\nWhat if the team misses to identify several generic components for which COTS are available ?\n\t\uf0e0the team develops everything in-house…\n\n\n"
each_assistant = "The PowerPoint presentation highlights several design flaws such as the shopping cart becoming common for multiple customers, the web server crashing due to excessive processing, hackers manipulating prices due to ignored security requirements, and developing everything in-house due to missed identification of available COTS components."
prompt_each = make_prompt_format(each_system, each_user, each_assistant)

result_each = {}
app_each = ChatApp(prompt_each)
for i in range(len(each_slide)):
# for i in range(5):
  message = each_slide[f'{i}']
  res = app_each.chat(message)
  result_each[f'{i}'] = res

result_each

{'0': 'Architectural design refers to the process of defining the structure, behavior, and components of a software system. It involves identifying the various modules of the system, their relationships, and how they interact with each other. The architectural design is crucial as it lays the foundation for the rest of the software development process. A good architectural design ensures that the software system is scalable, maintainable, and meets the functional and non-functional requirements. It also helps in identifying potential risks and mitigating them in the early stages of development.',
 '1': "The PowerPoint presentation explains that architectural design is crucial as it identifies the main components and their relationships in a software system. The output of this phase is an architectural model that determines the system's organization as a set of communicating components. The architecture of a software system has a significant impact on its performance, efficiency, securi

##Keyword

In [35]:
#prompt demonstration
keyword_system = "I want you to act as a keyword detector. I will give you the contents of a PowerPoint presentation, and you will provide me with the relevant keywords along with their explanations."
keyword_user = "slide 16: Agility & Architecture Agile approach implements a few requirements in each sprint. However, we cannot do the same for the architecture of a complex system! We cannot start with the bare-minimum architecture & keep expanding.  Refactoring/ revamping the architecture may be as bad as restarting from scratch – can be expensive & time-consuming!  In other words, agile approach is primarily for development & beyond…"
keyword_assistant = "slide: 16: 1. Agility: the ability of a team or organization to quickly adapt to changing circumstances, requirements, or priorities. It focuses on being flexible, responsive, and adaptive in the face of uncertainties, evolving customer needs, and market conditions. 2. Agile approach: a project management and product development methodology that emphasizes iterative and incremental progress, collaboration, and flexibility. In Agile, work is organized into small, manageable tasks called sprints, which are typically two to four weeks in duration. Teams prioritize and complete these tasks while regularly reassessing and adjusting their priorities based on feedback and changing requirements. 3. Bare-minimum architecture: the simplest possible structure or design for a system that can still meet its basic functional requirements. This approach is usually not recommended for complex systems, as it can lead to significant challenges when it comes to expanding or refining the architecture to accommodate new features or improvements. 4. Refactoring: the process of restructuring and improving existing code without changing its external behavior or functionality. This can involve cleaning up the code, optimizing performance, or simplifying the design to make it more maintainable and easier to understand. 5. Revamping: the process of making significant improvements or overhauling an existing system, product, or process. In the context of software architecture, this might involve rethinking the overall design, updating outdated components, or introducing new technologies to better align with current requirements and best practices. 6. Scratch: In the context of software development, \"starting from scratch\" refers to the act of creating a new system, application, or component entirely from the beginning, rather than building upon or modifying an existing one. This can be a time-consuming and costly process, but in some cases, it may be necessary if the existing system or architecture is deemed unsuitable or too difficult to refactor or revamp."
prompt_keyword = make_prompt_format(keyword_system, keyword_user, keyword_assistant)

result_keyword = {}
for i in range(len(each_slide)):
# for i in range(2):
  app_keyword = ChatApp(prompt_keyword)
  message = each_slide[f'{i}']
  res = app_keyword.chat(message)
  result_keyword[f'{i}'] = res

result_keyword

{'0': 'I can provide you with the relevant keywords and their explanations related to architectural design. Please let me know if you want me to proceed.',
 '1': "Keywords and their explanations:\n\n1. Architectural design: the process of defining the overall structure, composition, behavior, and functionality of a software system. It involves identifying the key components, interfaces, and relationships between different parts of the system, as well as specifying the constraints, requirements, and quality attributes that the architecture must satisfy. \n\n2. Structural components: the primary building blocks or modules of a software system, such as classes, objects, functions, or services. These components may interact with each other through well-defined interfaces, protocols, or APIs, and may be organized hierarchically or functionally based on their roles and responsibilities. \n\n3. Architectural model: a high-level abstraction or representation of the software system's architectu